In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup as bs
import pandas as pd

# Overview
- In this tutorial we will be going through how to scrape a Youtube Channel to get the latest posts and when they posted using selenium

## First we going to get the youtube url

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.youtube.com/c/steezystudio/videos')



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\kyled\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


## Scrape
+ Now that we have the youtube channel in memory let's get the information needed:
    - Title
    - Link
    - Views
    - Posted

In [3]:
for _ in range(10):
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    driver.implicitly_wait(2)
html = driver.page_source
soup = bs(html, 'html.parser')

In [4]:
videos = soup.find_all('div', {'id':'dismissible'})
youtube_list = []
for vid in videos:
    datadict = {}
    datadict['Title'] = vid.find('a', {'id':'video-title'}).text
    datadict['Link'] = 'https://www.youtube.com'+vid.find('a', {'id':'video-title'})['href']
    meta = vid.find('div', {'id':'metadata-line'}).find_all('span')
    datadict['Views'] = meta[0].text
    datadict['Posted'] = meta[1].text
    youtube_list.append(datadict)

## Dataframe with Pandas
- Once we have exactly what we need, let's create a dataframe in a tabular form

In [5]:
youtube = pd.DataFrame(youtube_list)
youtube

,Title,Link,Views,Posted
0,dancing with the boiz | STEEZY.CO #Shorts,https://www.youtube.com/watch?v=9P4lQ0J6MZM,30K views,2 days ago
1,Body - Megan Thee Stallion | Jonathan Sison Ch...,https://www.youtube.com/watch?v=OsXsz_9oPNU,10K views,3 days ago
2,Beginner vs Advanced Dancer – 2 Levels of Danc...,https://www.youtube.com/watch?v=sLGGIqsf98c,30K views,1 week ago
3,How to behave in a dance class | STEEZY.CO #Sh...,https://www.youtube.com/watch?v=p79XhmG-Kj8,15K views,1 week ago
4,Brinn Nicole making dancer sounds | STEEZY.CO ...,https://www.youtube.com/watch?v=ZcszW6URIl4,12K views,1 week ago
...,...,...,...,...
205,Stage Formations That Killer Dance Teams Alway...,https://www.youtube.com/watch?v=UVnZNm8OVxY,50K views,1 year ago
206,How A Professional Dancer Choreographs A Dance...,https://www.youtube.com/watch?v=qQbnXt0VJuE,420K views,1 year ago
207,Juicy - Doja Cat | Nicole Kirkland Choreograph...,https://www.youtube.com/watch?v=R1aDqzCAd80,180K views,1 year ago
208,"STEEZY Responds To Your Top Comments - Tips, A...",https://www.youtube.com/watch?v=-izbOTo6BFE,24K views,1 year ago


# Cleaning
- Although we have a good dataset let's clean the Views column

In [6]:
youtube.select_dtypes(object)

,Title,Link,Views,Posted
0,dancing with the boiz | STEEZY.CO #Shorts,https://www.youtube.com/watch?v=9P4lQ0J6MZM,30K views,2 days ago
1,Body - Megan Thee Stallion | Jonathan Sison Ch...,https://www.youtube.com/watch?v=OsXsz_9oPNU,10K views,3 days ago
2,Beginner vs Advanced Dancer – 2 Levels of Danc...,https://www.youtube.com/watch?v=sLGGIqsf98c,30K views,1 week ago
3,How to behave in a dance class | STEEZY.CO #Sh...,https://www.youtube.com/watch?v=p79XhmG-Kj8,15K views,1 week ago
4,Brinn Nicole making dancer sounds | STEEZY.CO ...,https://www.youtube.com/watch?v=ZcszW6URIl4,12K views,1 week ago
...,...,...,...,...
205,Stage Formations That Killer Dance Teams Alway...,https://www.youtube.com/watch?v=UVnZNm8OVxY,50K views,1 year ago
206,How A Professional Dancer Choreographs A Dance...,https://www.youtube.com/watch?v=qQbnXt0VJuE,420K views,1 year ago
207,Juicy - Doja Cat | Nicole Kirkland Choreograph...,https://www.youtube.com/watch?v=R1aDqzCAd80,180K views,1 year ago
208,"STEEZY Responds To Your Top Comments - Tips, A...",https://www.youtube.com/watch?v=-izbOTo6BFE,24K views,1 year ago


In [7]:
def convert_views(df):
    if 'K' in df['Views']:
        views = float(df['Views'].split('K')[0]) * 1000
        return views
    elif 'M' in df['Views']:
        views = float(df['Views'].split('M')[0]) * 1000000
        return views
    else:
        views = df['Views'].replace('views', '')
        return views

youtube['Clean_Views'] = youtube.apply(convert_views, axis=1)

In [8]:
youtube['Clean_Views'] = youtube['Clean_Views'].astype(int)
youtube.to_csv('Youtube_Channel.csv')
youtube.head(10)

,Title,Link,Views,Posted,Clean_Views
0,dancing with the boiz | STEEZY.CO #Shorts,https://www.youtube.com/watch?v=9P4lQ0J6MZM,30K views,2 days ago,30000
1,Body - Megan Thee Stallion | Jonathan Sison Ch...,https://www.youtube.com/watch?v=OsXsz_9oPNU,10K views,3 days ago,10000
2,Beginner vs Advanced Dancer – 2 Levels of Danc...,https://www.youtube.com/watch?v=sLGGIqsf98c,30K views,1 week ago,30000
3,How to behave in a dance class | STEEZY.CO #Sh...,https://www.youtube.com/watch?v=p79XhmG-Kj8,15K views,1 week ago,15000
4,Brinn Nicole making dancer sounds | STEEZY.CO ...,https://www.youtube.com/watch?v=ZcszW6URIl4,12K views,1 week ago,12000
5,If Brian Puspos danced to IceJJFish | STEEZY.C...,https://www.youtube.com/watch?v=lfAT3xuLEJ0,11K views,1 week ago,11000
6,Dancer Talk with Nicole Kirkland | STEEZY #Shorts,https://www.youtube.com/watch?v=ivzPBXMEXgw,11K views,1 week ago,11000
7,Can you name these dance moves? | STEEZY #Shorts,https://www.youtube.com/watch?v=0z9TGLMVee4,8K views,2 weeks ago,8000
8,Ultimate Guide To Learning Dance For Beginners...,https://www.youtube.com/watch?v=C62596gVnT8,27K views,2 weeks ago,27000
9,What’s That Dance Move? | STEEZY.CO #Shorts,https://www.youtube.com/watch?v=fi9rbSSFHIc,8.8K views,2 weeks ago,8800
